# SimResults To XWiki Tables - Race Reports and Championship Standings

In [99]:
import pandas as pd
import numpy as np
import re, os

### Race Report Generator

In [100]:
class RaceReport():
    
    # Race report table format strings
    qualy_row_0 = """(% border="1" style="width:554px" %)"""
    qualy_row_1 = """|=(% scope="row" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 41px; text-align: center;" %)Pos|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)No|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 155px; text-align: center;" %)Driver|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 233px; text-align: center;" %)Team|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 77px; text-align: center;" %)Time"""
    qualy_row_2 = """|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="border-color:#000000; text-align:center; width:44px" %){}|(% style="border-color:#000000; width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; border-color:#000000; width:233px" %){}|(% style="text-align:center; border-color:#000000; width:77px" %){}"""
    qualy_row = """|=(% style="background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="text-align:center; width:44px" %){}|(% style="width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; width:233px" %){}|(% style="text-align:center; width:77px" %){}"""
    qualy_row_last = """|=(% colspan="5" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 552px;" %)[[Source>>{}]]"""
    race_row_0 = """(% border="1" style="width:747px" %)"""
    race_row_1 = """|=(% scope="row" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 41px; text-align: center;" %)Pos|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)No|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 155px; text-align: center;" %)Driver|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 229px; text-align: center;" %)Team|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)Laps|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 111px; text-align: center;" %)Time/Retired|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 53px; text-align: center;" %)Grid|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 62px; text-align: center;" %)Points"""
    race_row_2 = """|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="border-color:#000000; text-align:center; width:44px" %){}|(% style="border-color:#000000; width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; border-color:#000000; width:233px" %){}|(% style="text-align:center; border-color:#000000; width:44px" %){}|(% style="text-align:center; border-color:#000000; width:111px" %){}|(% style="text-align:center; border-color:#000000; width:53px" %){}|(% style="text-align:center; border-color:#000000; width:62px" %){}"""
    race_row = """|=(% style="background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="text-align:center; width:44px" %){}|(% style="width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; width:233px" %){}|(% style="text-align:center; width:44px" %){}|(% style="text-align:center; width:111px" %){}|(% style="text-align:center; width:53px" %){}|(% style="text-align:center; width:62px" %){}"""
    race_row_last = """|=(% colspan="8" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 745px;" %)[[Source>>{}]]"""
    race_row_fastestlap = """Fastest lap:  [[image:{}||height="14" width="23"]] {} - {}"""
    
    
    # Read series drivers info
    def read_drivers_table(series_directory):
        drivers_table_file = "{}/drivers_table.csv".format(series_directory)
        table = pd.read_csv(drivers_table_file, dtype=object)
        table = table.set_index("ign")
        return table
    
    
    # Read series points scoring info
    def read_points_table(series_directory):
        points_table_file = "{}/points_table.csv".format(series_directory)
        table = pd.read_csv(points_table_file, dtype=object)
        table = table.astype(int)
        table = table.set_index("pos")
        return table
    
    
    # Read series track info (used only for championship table)
    def read_tracks_table(series_directory):
        tracks_table_file = "{}/tracks_table.csv".format(series_directory)
        table = pd.read_csv(tracks_table_file, dtype=object)
        table["csv_manual_adjustment"] = table["csv_manual_adjustment"].astype(int)
        table = table.set_index("directory")
        return table
    
    
    # Constructor, optionally pass in already-parsed drivers and points table info, debug to print intermediate dataframes
    def __init__(self, table_names, series_directory, race_directory, drivers_table=None, points_table=None, csv_manual_adjustment=0, output_file_name="wiki_tables.txt", debug=False):
        
        self.table_names = table_names
        self.debug = debug
        
        if self.debug: print("Creating race report:", series_directory, race_directory)
        
        self.drivers_table = drivers_table if drivers_table is not None else RaceReport.read_drivers_table(series_directory)
        self.points_table = points_table if points_table is not None else RaceReport.read_points_table(series_directory)

        race_directory_path = "{}/{}".format(series_directory, race_directory)
        assert os.path.isdir(race_directory_path), "Race directory path does not exist: {}".format(race_directory_path)
        
        self.output_file = "{}/{}".format(race_directory_path, output_file_name)
        
        csv_files = [file for file in os.listdir(race_directory_path) if file.endswith(".csv")]
        if len(csv_files) < 1:
            raise FileNotFoundError()
        simresults_file_name = [file for file in os.listdir(race_directory_path) if file.endswith(".csv")][0]
        simresults_code = os.path.splitext(simresults_file_name)[0]
        self.results_file = "{}/{}.csv".format(race_directory_path, simresults_code)
        self.simresults_url = "https://simresults.net/{}".format(simresults_code)
        
        self.csv_manual_adjustment = csv_manual_adjustment
        self.tables = self.__read_results_tables()
        self.__clean_results_tables()
    
    
    # Return race result pandas dataframes in dict keyed by table_names
    def __read_results_tables(self):
        
        rows = {name: [0,0] for name in self.table_names}
        with open(self.results_file) as fp:
            current_table = ""
            for i, row in enumerate(fp):
                if current_table:
                    if row=="\n":
                        rows[current_table][1] = i
                        if self.debug: print("ending line {}".format(rows[current_table][1]))
                        current_table = ""
                else:
                    for name in self.table_names:
                        if row.startswith(name):
                            current_table = name
                            rows[current_table][0] = i+2
                            if self.debug: print("found table '{}' starting line {} ".format(current_table, rows[current_table][0]), end="")
        
        tables = {}
        for name in self.table_names:
            table_skiprows = rows[name][0] + self.csv_manual_adjustment
            table_nrows = rows[name][1] - table_skiprows - 1 + self.csv_manual_adjustment
            table_df = pd.read_csv(self.results_file, skiprows=table_skiprows, nrows=table_nrows, index_col=False, dtype=object)
            if self.debug: display(table_df)
            tables[name] = table_df
        
        return tables
    
    
    # Clean race result dataframes, cast numerical columns to integers, join points information and starting positions to driver rows
    def __clean_results_tables(self):
        
        # Merge starting position info to grid column of a race table from either quali session or previous race, optionally adding quali points column
        def merge_qualy_info(race_table, qualy_table, add_qualy_points=False):
            
            qualy_table = qualy_table.reset_index()
            race_table = race_table.reset_index()
            
            if add_qualy_points:
                qualy_table = qualy_table[["Pos", "Driver", "Points"]]
                qualy_table = qualy_table.rename(columns={"Pos":"Grid", "Points":"Qualify Points"})
            else:
                qualy_table = qualy_table[["Pos", "Driver"]]
                qualy_table = qualy_table.rename(columns={"Pos":"Grid"})

            race_table = race_table.merge(qualy_table, how='left', on="Driver")

            race_table["Grid"] = race_table["Grid"].fillna(-1)
            # Pandas issue where NaN values cause ints to become floats?
            race_table["Grid"] = race_table["Grid"].astype(int)
            race_table["Pos"] = race_table["Pos"].astype(int)
            race_table["Laps"] = race_table["Laps"].astype(int)

            if add_qualy_points:
                race_table["Qualify Points"] = race_table["Qualify Points"].fillna(0)
                race_table["Qualify Points"] = race_table["Qualify Points"].astype(int)
                
            race_table = race_table.set_index("Pos")

            return race_table
        
        # Strip quotes and whitespace from strings, cast position and laps to integers, drop unnamed columns and rows for non-participants, convert fastest laps to datetimes
        for name, table_df in self.tables.items():

            table_df = table_df.apply(lambda s: s.str.strip(' \'"'), axis=1)
            table_df = table_df.rename(columns=lambda c: c.strip(' \'"'))
            table_df = table_df.apply(lambda s: s.str.replace('(.*\d\d*\.\d{3})0$', r'\1'), axis=1)

            table_df["Pos"] = table_df["Pos"].astype(int)
            table_df = table_df.set_index("Pos")
            
            table_df["Laps"] = table_df["Laps"].astype(int)

            table_df = table_df[table_df["Laps"] > 0]

            table_df = table_df[table_df.columns.drop(list(table_df.filter(regex='Unnamed*')))]
            table_df = table_df.dropna(how='all', axis='columns')
            
            table_df["Best lap time"] = pd.to_datetime(table_df["Best lap"], format="%M:%S.%f", errors='coerce')

            self.tables[name] = table_df
            
            
        # Merge points column to both quali and race sessions, for race sessions: get attatched qualifying or previous race to get starting positions and show (probable) DNFs
        for name, table_df in self.tables.items():

            if name.startswith("Qualify"):

                points_column = pd.Series(np.zeros(len(table_df))).add(self.points_table["qualy_points"].astype(int), fill_value=0)
                table_df["Points"] = points_column.astype(int)

            if name.startswith("Race"):

                table_df["Time/Retired"] = table_df["Time/Retired"].str.replace('^\s*$', "")
                table_df["Consistency"] = table_df["Consistency"].str.replace('^-$', "")

                points_column = pd.Series(np.zeros(len(table_df)+1)).add(self.points_table["points"].astype(int), fill_value=0)
                table_df["Points"] = points_column.astype(int)

                session_num = int(re.findall('(?:Qualify|Race)\s(\d+)\sresult|$', name)[0])
                qualy_table_name = "Qualify result" if session_num==1 and "Qualify result" in self.tables.keys() else "Qualify {} result".format(session_num)
                previous_race_table_name = "Race {} result".format(session_num - 1)
                if qualy_table_name in self.tables:
                    table_df = merge_qualy_info(table_df, self.tables[qualy_table_name], add_qualy_points=True)
                elif previous_race_table_name in self.tables:
                    table_df = merge_qualy_info(table_df, self.tables[previous_race_table_name])
                else:
                    table_df["Grid"] = ""
                    
                table_df["DNF"] = table_df["Laps"] < table_df["Laps"].max() - 3
                
            # Validate drivers are in the driver table
            for driver in table_df["Driver"]:
                assert driver in self.drivers_table.index, "{} not found in drivers table".format(driver)

            if self.debug: display(table_df)
            self.tables[name] = table_df
        

    # Generate quali table markdown from quali session dataframe, result is a list of lines
    def __generate_qualy_table_strings(self, table_name):

        table_df = self.tables[table_name]

        lines_buffer = [table_name, self.qualy_row_0, self.qualy_row_1]

        for i, df_row in table_df.iterrows():

            driver = df_row["Driver"]

            driver_info = self.drivers_table.loc[driver]

            position = i
            number = driver_info["number"]
            flag = driver_info["flag"]
            name = driver_info["name"]
            team = driver_info["team"]
            time = df_row["Best lap"]
            if position==1:
                time = "**{}**".format(time)

            line = self.qualy_row_2.format(position, number, flag, name, team, time) if i==0 else self.qualy_row.format(position, number, flag, name, team, time)
            lines_buffer.append(line)

        lines_buffer.append(self.qualy_row_last.format(self.simresults_url))

        if self.debug: print("made {} rows for table: {}".format(len(lines_buffer), table_name))
        return lines_buffer
    
    
    def __generate_race_table_strings(self, table_name):
    
        table_df = self.tables[table_name]

        lines_buffer = [table_name, self.race_row_0, self.race_row_1]

        for i, df_row in table_df.iterrows():

            driver = df_row["Driver"]

            driver_info = self.drivers_table.loc[driver]

            position = i
            number = driver_info["number"]
            flag = driver_info["flag"]
            name = driver_info["name"]
            team = driver_info["team"]
            laps = df_row["Laps"]
            points = df_row["Points"]
            timeorretired = df_row["Time/Retired"] if df_row["Time/Retired"] else "DNF"
            grid = df_row["Grid"] if df_row["Grid"]>0 else "DNQ"

            if "Qualify Points" in table_df.columns:
                points = str(df_row["Points"] + df_row["Qualify Points"]) + ("^^{}^^".format(grid) if df_row["Qualify Points"]>0 else "")

            line = self.race_row_2.format(position, number, flag, name, team, laps, timeorretired, grid, points) if i==0 else self.race_row.format(position, number, flag, name, team, laps, timeorretired, grid, points)
            lines_buffer.append(line)

        lines_buffer.append(self.race_row_last.format(self.simresults_url))
        
        fastest_driver = table_df.loc[table_df["Best lap time"]==table_df["Best lap time"].min()].iloc[0]["Driver"]
        fastest_driver_flag = self.drivers_table.loc[fastest_driver, "flag"]
        
        fastest_time = table_df.loc[table_df["Driver"]==fastest_driver]["Best lap"].item()
        lines_buffer.append(self.race_row_fastestlap.format(fastest_driver_flag, fastest_driver, fastest_time))

        if self.debug: print("made {} rows for table: {}".format(len(lines_buffer), table_name))
        return lines_buffer
    
    
    # Generate table markdown for all table_names, result is map from table names to table markdown string
    def generate_tables_strings(self):
        tables_strings = {}

        for name in self.table_names:
            lines_buffer = ["error!"]
            if name.startswith("Qualify"):
                lines_buffer = self.__generate_qualy_table_strings(name)
            elif name.startswith("Race"):
                lines_buffer = self.__generate_race_table_strings(name)
            tables_strings[name] = "\n".join(lines_buffer) + "\n\n"
            
        return tables_strings
    
    
    # Write table markdown for all table_names, optionally provide the generated table strings
    def write_generated_tables(self, tables_strings=None):
        if not tables_strings:
            tables_strings = self.generate_tables_strings()
            
        with open(self.output_file, "w+") as fp:
            for name in self.table_names:
                fp.write(tables_strings[name])
                if self.debug: print("wrote table {} to {}".format(name, self.output_file))

    

### Read Series Info

In [101]:
series = "MX5"
series_sessions = ["Qualify result", "Race 1 result", "Race 2 result"]
rounds_to_include = 4
drop_week = False

series_drivers_table = RaceReport.read_drivers_table(series)
display(series_drivers_table)

series_points_table = RaceReport.read_points_table(series)
display(series_points_table)

series_tracks_table = RaceReport.read_tracks_table(series)
display(series_tracks_table)

series_race_sessions = [session for session in series_sessions if session.startswith("Race")]
num_total_races = len(series_tracks_table) * len(series_race_sessions)

,name,number,team,flag
ign,,,,
Electro,Electro,25,Akrapovic GP,https://upload.wikimedia.org/wikipedia/commons...
gunciz,Gunciz,000,Bubonic Clutch Group,https://upload.wikimedia.org/wikipedia/commons...
maestro,Maestro,037,Bubonic Clutch Group,https://upload.wikimedia.org/wikipedia/en/thum...
swordhaven,Swordhaven,019,Croshaven,https://upload.wikimedia.org/wikipedia/commons...
Crosdale,Crosdale,100,Croshaven,https://upload.wikimedia.org/wikipedia/en/thum...
NihonTiger,NihonTiger,12,Element Zero Motorsports,https://upload.wikimedia.org/wikipedia/en/thum...
PhotonBurst,PhotonBurst,28,Element Zero Motorsports,https://upload.wikimedia.org/wikipedia/commons...
Viperion_NZ,Viperion_NZ,05,Kiwilight Racing,https://upload.wikimedia.org/wikipedia/commons...
Sakia,Sakia,404,Kiwilight Racing,https://upload.wikimedia.org/wikipedia/commons...


,points,qualy_points
pos,,
1,40,3
2,36,2
3,32,1
4,28,0
5,25,0
6,22,0
7,19,0
8,16,0
9,13,0


,abbrev,full_name,flag,csv_manual_adjustment
directory,,,,
donington,DON,Donington,https://upload.wikimedia.org/wikipedia/en/thum...,0
watkins,WAT,Watkins Glen,https://upload.wikimedia.org/wikipedia/en/thum...,0
knockhill,KNO,Knockhill,https://upload.wikimedia.org/wikipedia/en/thum...,0
zandvoort,ZAN,Zandvoort,https://upload.wikimedia.org/wikipedia/commons...,0
highplains,HPL,High Plains,https://upload.wikimedia.org/wikipedia/en/thum...,0
oulton,OUL,Oulton Park,https://upload.wikimedia.org/wikipedia/en/thum...,0
vir,VIR,Virginia International Raceway,https://upload.wikimedia.org/wikipedia/en/thum...,0
nurb,NUR,Nurburgring,https://upload.wikimedia.org/wikipedia/en/thum...,0


### Read Race Results

In [102]:
race_reports = {}

for race, race_row in series_tracks_table.iterrows():
    
    race_path = "{}/{}".format(series, race)
    if os.path.isdir(race_path):
        try:
            race_reports[race] = RaceReport(series_sessions, series, race, drivers_table=series_drivers_table, points_table=series_points_table, csv_manual_adjustment=race_row["csv_manual_adjustment"])
            print(race)
            [display(race_reports[race].tables[session]) for session in series_race_sessions]
        except FileNotFoundError:
            print("no csv found for", race)
    else:
        print("no directory found for", race)

donington


,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,Qualify Points,DNF
Pos,,,,,,,,,,,,,,,
1,,,ks_mazda_mx5_cup,PhotonBurst,12,15:44.008,01:17.710,99.35%,11,0,1900-01-01 00:01:17.710,40,1,3,False
2,,,ks_mazda_mx5_cup,sharkeh,12,+00:01.562,01:17.844,99.41%,0,0,1900-01-01 00:01:17.844,36,2,2,False
3,,,ks_mazda_mx5_cup,JEMknight657,12,+00:01.765,01:17.736,99.27%,0,0,1900-01-01 00:01:17.736,32,3,1,False
4,,,ks_mazda_mx5_cup,MrGrinder,12,+00:02.857,01:17.849,99.32%,0,0,1900-01-01 00:01:17.849,28,5,0,False
5,,,ks_mazda_mx5_cup,McLarenTim,12,+00:06.098,01:18.274,99.67%,0,0,1900-01-01 00:01:18.274,25,4,0,False
6,,,ks_mazda_mx5_cup,Electro,12,+00:12.072,01:18.263,99.29%,0,0,1900-01-01 00:01:18.263,22,6,0,False
7,,,ks_mazda_mx5_cup,Trempale,12,+00:12.230,01:18.367,99.21%,0,0,1900-01-01 00:01:18.367,19,9,0,False
8,,,ks_mazda_mx5_cup,NotJames,12,+00:22.112,01:18.896,98.89%,0,0,1900-01-01 00:01:18.896,16,13,0,False
9,,,ks_mazda_mx5_cup,maestro,12,+00:22.379,01:18.789,98.59%,0,0,1900-01-01 00:01:18.789,13,11,0,False


,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,DNF
Pos,,,,,,,,,,,,,,
1,,,ks_mazda_mx5_cup,PhotonBurst,20,26:06.445,01:17.720,99.60%,19,0,1900-01-01 00:01:17.720,40,1,False
2,,,ks_mazda_mx5_cup,sharkeh,20,+00:01.830,01:17.765,99.58%,0,0,1900-01-01 00:01:17.765,36,2,False
3,,,ks_mazda_mx5_cup,JEMknight657,20,+00:08.216,01:17.772,99.21%,0,0,1900-01-01 00:01:17.772,32,3,False
4,,,ks_mazda_mx5_cup,Electro,20,+00:13.320,01:18.105,99.40%,0,0,1900-01-01 00:01:18.105,28,6,False
5,,,ks_mazda_mx5_cup,MrGrinder,20,+00:22.896,01:18.192,98.72%,0,0,1900-01-01 00:01:18.192,25,4,False
6,,,ks_mazda_mx5_cup,gunciz,20,+00:22.969,01:18.155,98.91%,0,0,1900-01-01 00:01:18.155,22,10,False
7,,,ks_mazda_mx5_cup,maestro,20,+00:24.992,01:18.523,99.17%,0,0,1900-01-01 00:01:18.523,19,9,False
8,,,ks_mazda_mx5_cup,Georgin,20,+00:38.316,01:19.114,99.18%,0,0,1900-01-01 00:01:19.114,16,11,False
9,,,ks_mazda_mx5_cup,McLarenTim,20,+00:39.154,01:18.272,97.80%,0,0,1900-01-01 00:01:18.272,13,5,False


watkins


,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,Qualify Points,DNF
Pos,,,,,,,,,,,,,,,
1,,,ks_mazda_mx5_cup,NotJames,11,15:55.761,01:25.408,99.02%,8,0,1900-01-01 00:01:25.408,40,4,0,False
2,,,ks_mazda_mx5_cup,Waffles,11,+00:03.138,01:25.472,98.67%,2,0,1900-01-01 00:01:25.472,36,5,0,False
3,,,ks_mazda_mx5_cup,NihonTiger,11,+00:03.569,01:25.797,99.17%,0,0,1900-01-01 00:01:25.797,32,1,3,False
4,,,ks_mazda_mx5_cup,Salzeder,11,+00:05.055,01:25.725,99.00%,0,0,1900-01-01 00:01:25.725,28,6,0,False
5,,,ks_mazda_mx5_cup,Georgin,11,+00:06.342,01:25.773,98.98%,0,0,1900-01-01 00:01:25.773,25,3,1,False
6,,,ks_mazda_mx5_cup,Smartpipe,11,+00:09.047,01:26.032,99.09%,0,0,1900-01-01 00:01:26.032,22,8,0,False
7,,,ks_mazda_mx5_cup,maestro,11,+00:09.901,01:25.974,99.03%,0,0,1900-01-01 00:01:25.974,19,10,0,False
8,,,ks_mazda_mx5_cup,Crosdale,11,+00:11.835,01:25.983,98.53%,0,0,1900-01-01 00:01:25.983,16,7,0,False
9,,,ks_mazda_mx5_cup,Horanjo,11,+00:19.183,01:26.545,98.52%,0,0,1900-01-01 00:01:26.545,13,13,0,False


...

,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,DNF
Pos,,,,,,,,,,,,,,
1,,,ks_mazda_mx5_cup,MrGrinder,26,25:29.859,00:57.931,98.91%,24,0,1900-01-01 00:00:57.931,40,6,False
2,,,ks_mazda_mx5_cup,gunciz,26,+00:16.126,00:57.891,97.71%,0,0,1900-01-01 00:00:57.891,36,4,False
3,,,ks_mazda_mx5_cup,Waffles,26,+00:16.203,00:58.402,98.71%,0,0,1900-01-01 00:00:58.402,32,8,False
4,,,ks_mazda_mx5_cup,PhotonBurst,26,+00:22.089,00:57.913,97.66%,0,0,1900-01-01 00:00:57.913,28,1,False
5,,,ks_mazda_mx5_cup,xPsychedelix,26,+00:25.487,00:58.324,97.94%,0,0,1900-01-01 00:00:58.324,25,7,False
6,,,ks_mazda_mx5_cup,Trempale,26,+00:26.682,00:57.832,97.59%,0,0,1900-01-01 00:00:57.832,22,3,False
7,,,ks_mazda_mx5_cup,Salzeder,26,+00:32.968,00:58.671,98.07%,0,0,1900-01-01 00:00:58.671,19,9,False
8,,,ks_mazda_mx5_cup,NihonTiger,26,+00:33.778,00:58.959,98.64%,0,0,1900-01-01 00:00:58.959,16,10,False
9,,,ks_mazda_mx5_cup,maestro,26,+00:38.930,00:58.541,97.29%,0,0,1900-01-01 00:00:58.541,13,5,False


zandvoort


,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,Qualify Points,DNF
Pos,,,,,,,,,,,,,,,
1,,,ks_mazda_mx5_cup,JEMknight657,8,15:46.373,01:57.240,99.76%,7,0,1900-01-01 00:01:57.240,40,2,2,False
2,,,ks_mazda_mx5_cup,Electro,8,+00:06.987,01:57.839,99.49%,0,0,1900-01-01 00:01:57.839,36,1,3,False
3,,,ks_mazda_mx5_cup,Waffles,8,+00:07.639,01:57.850,99.55%,0,0,1900-01-01 00:01:57.850,32,3,1,False
4,,,ks_mazda_mx5_cup,Georgin,8,+00:08.708,01:57.987,99.72%,0,0,1900-01-01 00:01:57.987,28,6,0,False
5,,,ks_mazda_mx5_cup,maestro,8,+00:08.826,01:57.492,99.30%,0,0,1900-01-01 00:01:57.492,25,8,0,False
6,,,ks_mazda_mx5_cup,McLarenTim,8,+00:09.075,01:57.042,98.62%,0,0,1900-01-01 00:01:57.042,22,4,0,False
7,,,ks_mazda_mx5_cup,Salzeder,8,+00:09.342,01:57.831,99.56%,0,0,1900-01-01 00:01:57.831,19,5,0,False
8,,,ks_mazda_mx5_cup,NotJames,8,+00:12.218,01:57.747,99.19%,0,0,1900-01-01 00:01:57.747,16,7,0,False
9,,,ks_mazda_mx5_cup,NihonTiger,8,+00:13.129,01:57.977,99.46%,0,0,1900-01-01 00:01:57.977,13,10,0,False


,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,DNF
Pos,,,,,,,,,,,,,,
1,,,ks_mazda_mx5_cup,JEMknight657,13,25:31.676,01:56.894,99.64%,12,0,1900-01-01 00:01:56.894,40,1,False
2,,,ks_mazda_mx5_cup,Waffles,13,+00:18.681,01:57.612,99.39%,0,0,1900-01-01 00:01:57.612,36,3,False
3,,,ks_mazda_mx5_cup,NotJames,13,+00:22.954,01:57.951,99.19%,0,0,1900-01-01 00:01:57.951,32,8,False
4,,,ks_mazda_mx5_cup,Electro,13,+00:23.458,01:57.345,98.30%,0,0,1900-01-01 00:01:57.345,28,2,False
5,,,ks_mazda_mx5_cup,Georgin,13,+00:24.846,01:57.112,98.16%,0,0,1900-01-01 00:01:57.112,25,4,False
6,,,ks_mazda_mx5_cup,McLarenTim,13,+00:26.017,01:57.387,98.28%,0,0,1900-01-01 00:01:57.387,22,6,False
7,,,ks_mazda_mx5_cup,maestro,13,+00:27.006,01:57.799,98.66%,0,0,1900-01-01 00:01:57.799,19,5,False
8,,,ks_mazda_mx5_cup,NihonTiger,13,+00:27.853,01:57.335,98.31%,0,0,1900-01-01 00:01:57.335,16,9,False
9,,,ks_mazda_mx5_cup,gunciz,13,+00:31.027,01:58.011,98.78%,0,0,1900-01-01 00:01:58.011,13,11,False


no csv found for highplains
no csv found for oulton
no csv found for vir
no csv found for nurb


### Construct Points Table

In [103]:
# compressed race result info obj for a driver and session

class DriverRaceResultInfo:
    
    def __init__(self, pos=-1, points=0, qualy_pos=-1, qualy_points=0, dnf=False):
        self.pos = pos
        self.points = points
        self.qualy_pos = qualy_pos
        self.qualy_points = qualy_points
        self.dnf = dnf
        
    def get_total_points(self):
        return self.points + self.qualy_points
        
    def __str__(self):
        if (self.pos<0) and (self.qualy_pos<0):
            return "NP"
        elif self.dnf:
            return "DNF"
        else:
            return str(self.get_total_points())
        
    def __repr__(self):
        return "DriverRaceResultInfo({},{},{},{},{})".format(self.pos, self.points, self.qualy_pos, self.qualy_points, self.dnf)
    

In [104]:
points_table = pd.DataFrame(index=series_drivers_table.index, columns=pd.MultiIndex.from_product([series_tracks_table.index, series_race_sessions], names=["track", "session"]))

# loop through drivers index
# construct series with the multindex
# go thorugh race reports and count up points

for driver in points_table.index:
    for track, session in points_table.columns:
        
        result_info = DriverRaceResultInfo()

        if track in race_reports:
            
            race_table = race_reports[track].tables[session]
            driver_race_result = race_table[race_table["Driver"]==driver]

            if len(driver_race_result==1):
                pos = driver_race_result.index[0]
                points = driver_race_result["Points"].item()
                qualify_pos = driver_race_result["Grid"].item()
                qualify_points = driver_race_result["Qualify Points"].item() if "Qualify Points" in driver_race_result.columns else 0
                dnf = driver_race_result["DNF"].item()
                
                result_info = DriverRaceResultInfo(pos, points, qualify_pos, qualify_points, dnf)
        
        points_table.loc[driver, (track, session)] = result_info
        
# truncate results to only calculate up to rounds needed

points_table = points_table.iloc[:, 0:(rounds_to_include*len(series_race_sessions))]

In [105]:
# keep seperate table with points totals

tracks = points_table.columns.unique(level="track")
points_totals_table = pd.DataFrame(index=series_drivers_table.index, columns=tracks)

def get_total_weekend_points(driver_session_results):
    return driver_session_results.apply(DriverRaceResultInfo.get_total_points).sum()

for track in tracks:
    weekend_points = points_table[track]
    points_totals_table[track] = weekend_points.apply(get_total_weekend_points, axis=1)

points_totals_table["total"] = points_totals_table.agg(sum, axis=1)
points_totals_table["total_with_drop_week"] = points_totals_table.apply(lambda driver_row: driver_row["total"] - (min(driver_row)), axis=1)

if drop_week:
    points_totals_table = points_totals_table.sort_values("total_with_drop_week", ascending=False)
else:
    points_totals_table = points_totals_table.sort_values("total", ascending=False)
points_table = points_table.reindex(points_totals_table.index)

display(points_table)
display(points_totals_table)

track              donington                     watkins                \
session        Race 1 result Race 2 result Race 1 result Race 2 result   
ign                                                                      
PhotonBurst               43            40             8             0   
maestro                   13            19            19            25   
Waffles                   NP            NP            36             0   
JEMknight657              33            32            NP            NP   
gunciz                    10            22            10            16   
Salzeder                   1            10            28            40   
NihonTiger                 4             4            35            36   
MrGrinder                 28            25             0             8   
Electro                   22            28             1           DNF   
Georgin                    8            16            26             0   
NotJames                  16             0            40             4   
Trempale                  19             8             6            19   
McLarenTim                25            13             4            13   
sharkeh                   38            36            NP            NP   
Smartpipe                  2             6            22            10   
xPsychedelix               6             2             0             6   
Crosdale                   0             1            16            32   
Viperion_NZ                0             0             0            28   
Baba                      NP            NP            NP            NP   
nemaides                  NP            NP             0            22   
Horanjo                    0             0            13             1   
marzen224                 NP            NP             0             0   
Monochromatica            NP            NP             4             2   
BarricadeTTT              NP            NP            NP            NP   
poppy s                   NP            NP            NP            NP   
gman197002                 0             0            NP            NP   
Sakia                      0           DNF            NP            NP   
swordhaven                 0             0            NP            NP   
mitski s                  NP            NP            NP            NP   

track              knockhill                   zandvoort                
session        Race 1 result Race 2 result Race 1 result Race 2 result  
ign                                                                     
PhotonBurst               43            28            10             1  
maestro                   25            13            25            19  
Waffles                   16            32            33            36  
JEMknight657              NP            NP            42            40  
gunciz                    30            36             8            13  
Salzeder                  13            19            19             8  
NihonTiger                10            16            13            16  
MrGrinder                 22            40             4             0  
Electro                   NP            NP            39            28  
Georgin                    6             6            28            25  
NotJames                  NP            NP            16            32  
Trempale                  32            22             0             0  
McLarenTim                NP            NP            22            22  
sharkeh                   NP            NP            NP            NP  
Smartpipe                  8            10            NP            10  
xPsychedelix              19            25             1             2  
Crosdale                   0             2            NP            NP  
Viperion_NZ                4           DNF             6             4  
Baba                      36             0            NP            NP  
nemaides                 

track,donington,watkins,knockhill,zandvoort,total,total_with_drop_week
ign,,,,,,
PhotonBurst,83,8,71,11,173,165
maestro,32,44,38,44,158,126
Waffles,0,36,48,69,153,153
JEMknight657,65,0,0,82,147,147
gunciz,32,26,66,21,145,124
Salzeder,11,68,32,27,138,127
NihonTiger,8,71,26,29,134,126
MrGrinder,53,8,62,4,127,123
Electro,50,1,0,67,118,118


### Print Driver's Standings

##### Header

In [106]:
pos_width = 55
driver_width = 170
result_width = 45
track_width = len(series_race_sessions)*result_width
points_width = 92
table_width = pos_width + driver_width + len(series_tracks_table)*track_width + points_width

In [107]:
header_0_format = """(% border="1" cellpadding="1" style="width:{table_width}px"%)"""
header_1_pos_driver_format = """|=(% colspan="1" rowspan="2" scope="row" style="border-color: rgb(0, 0, 0); text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width: {pos_width}px" %)Pos|=(% colspan="1" rowspan="2" style="border-color: rgb(0, 0, 0); text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width: {driver_width}px;" %)Driver"""
header_1_track_format = """|=(% colspan="2" rowspan="1" style="border-color: rgb(0, 0, 0); text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width: {track_width}px" %)((({header_1_track_flag_and_abbrev})))"""
header_1_track_flag_and_abbrev_format = """
[[image:{track_flag}||height="14" width="23"]]

{track_abbrev}
"""
header_1_points_format = """|=(% colspan="1" rowspan="2" style="border-color: rgb(0, 0, 0); text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width: {points_width}px" %)Points"""
header_2_session_format = """|(% style="background-color:#eaecf0; text-align:center; vertical-align:middle; width:{result_width}px" %)**{session_abbrev}**"""

In [108]:
result_color_top_3 = ["#ffffbf", "#dfdfdf", "#ffdf9f"] 
result_color_points = "#dfffdf"
result_color_ret = "#efcfff"
result_color_default = "#ffffff"

In [109]:
header_0 = header_0_format.format(table_width=table_width)

header_1_substrings = [header_1_pos_driver_format.format(pos_width=pos_width, driver_width=driver_width)]
for track in series_tracks_table.index:
    header_1_track_flag_and_abbrev = header_1_track_flag_and_abbrev_format.format(track_flag=series_tracks_table.loc[track]["flag"], track_abbrev=series_tracks_table.loc[track]["abbrev"])
    header_1_track = header_1_track_format.format(track_width=track_width, header_1_track_flag_and_abbrev=header_1_track_flag_and_abbrev)
    header_1_substrings.append(header_1_track)
header_1_substrings.append(header_1_points_format.format(points_width=points_width))
header_1 = "".join(header_1_substrings)

header_2_substrings = []
for _ in series_tracks_table.index:
    for session in series_race_sessions:
        session_number = re.findall('\d', session)[0]
        session_abbrev = "R{}".format(session_number)
        header_2_session = header_2_session_format.format(result_width=result_width, session_abbrev=session_abbrev)
        header_2_substrings.append(header_2_session)
header_2 = "".join(header_2_substrings)

lines_buffer = [header_0, header_1, header_2]

##### Driver Results

In [110]:
driver_row_pos_format = """|=(% style="text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width:{pos_width}px" %){pos}"""
driver_row_driver_format = """|(% style="width:{driver_width}px" %)[[image:{driver_flag}||height="14" width="23"]] {driver}"""
driver_row_result_format = """|(% style="background-color:{result_color}; text-align:center; vertical-align:middle; width:{result_width}px" %){result}"""
driver_row_points_format = """|(% style="text-align:center; vertical-align:middle; width:{points_width}px" %){points}"""

In [111]:
for pos, driver in enumerate(points_table.index):
    pos += 1
    
    driver_row_pos = driver_row_pos_format.format(pos_width=pos_width, pos=pos)
    
    driver_info = series_drivers_table.loc[driver]
    driver_flag = driver_info["flag"]
    driver_full_name = driver_info["name"]
    driver_row_driver = driver_row_driver_format.format(driver_width=driver_width, driver_flag=driver_flag, driver=driver_full_name)
    
    driver_row_substrings = [driver_row_pos, driver_row_driver]
    
    for i in range(num_total_races):
        
        result_string = ""
        result_color = result_color_default
        
        if i >= len(points_table.columns):
            driver_row_result = driver_row_result_format.format(result_color=result_color, result_width=result_width, result=result_string)
            driver_row_substrings.append(driver_row_result)
            continue
            
        result_info = points_table.loc[driver, points_table.columns[i]]
        
        if result_info.pos > 0:
            result_string = str(result_info.pos)
            if result_info.points > 0:
                result_color = result_color_points
            if result_info.pos <= 3:
                result_color = result_color_top_3[result_info.pos-1]
            if result_info.dnf:
                result_string = "RET"
                result_color = result_color_ret
            if result_info.qualy_points > 0:
                result_string = "{}^^{}^^".format(result_string, result_info.qualy_pos)
        
        driver_row_result = driver_row_result_format.format(result_color=result_color, result_width=result_width, result=result_string)
        driver_row_substrings.append(driver_row_result)
        
    driver_totals = points_totals_table.loc[driver]
    driver_total = driver_totals["total"]
    
    if driver_total == 0:
        continue
    
    if drop_week:
        driver_total_with_drop_week = driver_totals["total_with_drop_week"]
        points_string = "**{}**".format(driver_total_with_drop_week)
        if not driver_total_with_drop_week == driver_total:
            points_string = "{}^^{}^^".format(points_string, driver_totals["total"])
    else:
        points_string = "**{}**".format(driver_total)
            
    driver_row_points = driver_row_points_format.format(points_width=points_width, points=points_string)
    driver_row_substrings.append(driver_row_points)
    
    driver_row = "".join(driver_row_substrings)
    lines_buffer.append(driver_row)

In [112]:
output_file = "{}/drivers_standings.txt".format(series)

with open(output_file, "w+") as fp:
    table_string = "\n".join(lines_buffer) + "\n\n"
    fp.write(table_string)

### Debug

In [113]:
# test_series_directory = "MX5"
# test_race_directory = "donington"
# test_table_names = ["Qualify result", "Race 1 result", "Race 2 result"]

# testReport = RaceReport(test_table_names, test_series_directory, test_race_directory, csv_manual_adjustment=0, debug=True)
# # testReport.write_generated_tables()

In [114]:
# table_width = pos_width + driver_width + 1*track_width + points_width

# print(header_0.format(table_width=table_width))
# print(header_1_pos_driver.format(pos_width=pos_width, driver_width=driver_width), end="")
# header_zolder_track_flag_and_abbrev = header_track_flag_and_abbrev.format(track_flag="https://upload.wikimedia.org/wikipedia/commons/thumb/9/92/Flag_of_Belgium_%28civil%29.svg/23px-Flag_of_Belgium_%28civil%29.svg.png", track_abbrev="ZOL")
# print(header_1_track.format(track_width=track_width, header_track_flag_and_abbrev=header_zolder_track_flag_and_abbrev), end="")
# print(header_1_points.format(points_width=points_width))

# for session in series_race_sessions:
#     session_number = re.findall('\d', session)[0]
#     session_abbrev = "R{}".format(session_number)
#     print(header_2_session.format(result_width=result_width, session_abbrev=session_abbrev), end="")
    
# print()